In [78]:
import pandas as pd
import json
import numpy as np
from itertools import permutations
from scipy.spatial.distance import cdist
import pandas as pd
import numpy as np
from itertools import permutations
from math import sqrt
import random

In [79]:
# Parámetros globales
MAX_SHIFT_TIME = 6.0  # hours
UNLOAD_TIME = 0.5     # hours per polygon
LOAD_TIME = 0.5       # hours at HQ
TRUCK_CAPACITY = 100  # max number of plants (can be modified)
HQ_POLYGON = 18       # headquarters polygon

with open('../scheduling/MC_Supply_Chain_Solution.json', 'r') as f:
    data = json.load(f)
    demand_df = pd.DataFrame(data['best_solution'], columns=['day', 'polygon', 'specie', 'supplier', 'amount'])

polygon_df = pd.read_csv('../setup/ha.csv')

In [80]:
# Helper Functions
def euclidean_distance(p1, p2):
    return sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

def compute_distance_matrix(polygon_coords):
    polygons = polygon_coords['Poligono'].tolist()
    coords = polygon_coords.set_index('Poligono')[['X', 'Y']].to_dict('index')
    dist_matrix = {}
    for i in polygons:
        dist_matrix[i] = {}
        for j in polygons:
            dist_matrix[i][j] = euclidean_distance(list(coords[i].values()), list(coords[j].values()))
    return dist_matrix

In [81]:
# Ant Colony Optimization Classes
class Ant:
    def __init__(self, demand, distance_matrix, truck_capacity):
        self.demand = demand.copy()
        self.distance_matrix = distance_matrix
        self.truck_capacity = truck_capacity
        self.route = []
        self.time = 0.0
        self.distance = 0.0
        self.load = 0
        self.current_polygon = HQ_POLYGON

    def visit_polygon(self, polygon, demand_amount):
        travel_time = self.distance_matrix[self.current_polygon][polygon] / 30.0  # Assume truck speed = 30 units/hour
        self.time += travel_time + UNLOAD_TIME
        self.distance += self.distance_matrix[self.current_polygon][polygon]
        self.current_polygon = polygon
        self.route.append(polygon)
        self.load -= demand_amount

    def reload_at_hq(self):
        travel_time = self.distance_matrix[self.current_polygon][HQ_POLYGON] / 30.0
        self.time += travel_time + LOAD_TIME
        self.distance += self.distance_matrix[self.current_polygon][HQ_POLYGON]
        self.current_polygon = HQ_POLYGON
        self.load = self.truck_capacity
        self.route.append(HQ_POLYGON)

    def construct_solution(self, day_demand):
        self.load = self.truck_capacity
        remaining_demand = day_demand.copy()
        visited = set()
        while not remaining_demand.empty:
            candidates = remaining_demand.groupby('polygon')['amount'].sum().sort_values().index.tolist()
            moved = False
            for polygon in candidates:
                total_amount = remaining_demand[remaining_demand['polygon'] == polygon]['amount'].sum()
                if total_amount <= self.load:
                    self.visit_polygon(polygon, total_amount)
                    remaining_demand = remaining_demand[remaining_demand['polygon'] != polygon]
                    moved = True
                    break
            if not moved:
                self.reload_at_hq()
            if self.time >= MAX_SHIFT_TIME:
                break

class ACO:
    def __init__(self, n_ants, n_iterations, alpha, beta, evaporation_rate, distance_matrix, demand):
        self.n_ants = n_ants
        self.n_iterations = n_iterations
        self.alpha = alpha
        self.beta = beta
        self.evaporation_rate = evaporation_rate
        self.distance_matrix = distance_matrix
        self.pheromone = {i: {j: 1.0 for j in distance_matrix[i]} for i in distance_matrix}
        self.demand = demand

    def run(self, day_demand):
        best_distance = float('inf')
        best_route = None
        for _ in range(self.n_iterations):
            ants = [Ant(self.demand, self.distance_matrix, TRUCK_CAPACITY) for _ in range(self.n_ants)]
            for ant in ants:
                ant.construct_solution(day_demand)
                if ant.distance < best_distance and ant.time <= MAX_SHIFT_TIME:
                    best_distance = ant.distance
                    best_route = ant.route
            self.evaporate_pheromone()
        return best_route, best_distance

    def evaporate_pheromone(self):
        for i in self.pheromone:
            for j in self.pheromone[i]:
                self.pheromone[i][j] *= (1 - self.evaporation_rate)

In [82]:
day = 1
daily_demand = demand_df[demand_df["day"] == day]
distance_matrix = compute_distance_matrix(polygon_df)

aco = ACO(n_ants=10, n_iterations=50, alpha=1, beta=2, evaporation_rate=0.1,
          distance_matrix=distance_matrix, demand=daily_demand)
best_route, best_distance = aco.run(daily_demand)

print("Best Route:", best_route)
print("Best Distance:", best_distance)

Best Route: None
Best Distance: inf
